# Гибридные рекомендательные системы

In [43]:
from surprise import SVD, Dataset, Reader
from surprise import accuracy
from surprise import SVDpp
from surprise.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook

# Загрузка данных из CSV файлов
movies = pd.read_csv('../../src/MovieLens/movies.csv')
ratings = pd.read_csv('../../src/MovieLens/ratings.csv')
tags = pd.read_csv('../../src/MovieLens/tags.csv')
links = pd.read_csv('../../src/MovieLens/links.csv')

## Объединение данных о фильмах и рейтингах

In [44]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

### Подготовка данных к обучению и обучение модели `SVD`

In [51]:
trainset, testset = train_test_split(data, test_size=.15, random_state=42)

algoSVD = SVD()
algoSVD.fit(trainset)
test_predSVD = algoSVD.test(testset)
rmseSVD = accuracy.rmse(test_predSVD, verbose=True)
print(f"RMSE_SVD: {rmseSVD}")
print(accuracy.rmse(test_predSVD, verbose=True))

RMSE: 0.8683
RMSE_SVD: 0.8682860514438984
RMSE: 0.8683
0.8682860514438984


### Подготовка данных к обучению и обучение модели `SVDpp`

In [52]:
algoSVDpp = SVDpp()
algoSVDpp.fit(trainset)
test_predSVDpp = algoSVDpp.test(testset)
rmseSVDpp = accuracy.rmse(test_predSVDpp, verbose=True)
print(f"RMSE_SVDpp: {rmseSVDpp}")
print(accuracy.rmse(test_predSVDpp, verbose=True))


RMSE: 0.8585
RMSE_SVDpp: 0.8585038632190032
RMSE: 0.8585
0.8585038632190032


### Расчет оценок для фильмов, которые пользователь еще не смотрел, на основании лучшей модели по оценке `RMSE`

In [53]:
current_user_id = 2.0
user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()

scores = []
titles = []

for movie in movies_with_ratings.title.unique():
    if movie in user_movies:
        continue
        
    scores.append(algoSVD.predict(uid=current_user_id, iid=movie).est)
    titles.append(movie)

print(sorted(scores)[-10:])

[4.331761912214249, 4.3487464184876625, 4.361508042030071, 4.397565361714495, 4.407643732847146, 4.412338738035427, 4.412401982530755, 4.415904038347125, 4.433901724113903, 4.470461694190552]


### Подготовка данных и `CountVectorizer` и поиск `NearestNeighbors`

In [54]:
def change_string(s):
    return ' '.join(s.replace(' ', '').replace('-', '').split('|'))

movie_genres = [change_string(g) for g in movies.genres.values]

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(movie_genres)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

neigh = NearestNeighbors(n_neighbors=20, metric='euclidean') 
neigh.fit(X_train_tfidf)

test = change_string("Adventure|Comedy|Fantasy|Crime")
predict = count_vect.transform([test])
X_tfidf2 = tfidf_transformer.transform(predict)

res = neigh.kneighbors(X_tfidf2, return_distance=True)

movies_with_ratings.sort_values('timestamp', inplace=True)

title_genres = {}

for index, row in tqdm_notebook(movies.iterrows()):
    title_genres[row.title] = row.genres

C:\Users\avsip\AppData\Local\Temp\ipykernel_5196\319968114.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for index, row in tqdm_notebook(movies.iterrows()):


0it [00:00, ?it/s]

### Функция гибридной рекомендательной системы

In [67]:

def hybrid_recommend(user_id, algo):
    current_user_id = user_id
    user_movies = movies_with_ratings[movies_with_ratings.userId == current_user_id].title.unique()
    
    last_user_movie = user_movies[-1]
    
    movie_genres = title_genres[last_user_movie]
    movie_genres = change_string(movie_genres)

    predict = count_vect.transform([movie_genres])
    X_tfidf2 = tfidf_transformer.transform(predict)

    res = neigh.kneighbors(X_tfidf2, return_distance=True)
    
    movies_to_score = movies.iloc[res[1][0]].title.values

    scores = []
    titles = []

    for movie in movies_to_score:
        if movie in user_movies:
            continue

        scores.append(algo.predict(uid=current_user_id, iid=movie).est)
        titles.append(movie)
    
    best_indexes = np.argsort(scores)[-10:]
    for i in reversed(best_indexes):
        print(f"{titles[i]:<50} {scores[i]:.2f}")


### Рекомендация фильмов для определенного `user`

In [68]:
user = 5
movies_with_ratings[movies_with_ratings.userId == user].sort_values('rating')

print("Вывод 10 рекомендуемых фильмов на основании алгоритма SVD и NearestNeighbors (гибридная система)\n")
hybrid_recommend(user, algoSVD)

Вывод 10 рекомендуемых фильмов на основании алгоритма SVD и NearestNeighbors (гибридная система)

Enemy of the State (1998)                          3.54
Assault on Precinct 13 (1976)                      3.53
Assignment, The (1997)                             3.53
Air Force One (1997)                               3.46
Payback (1999)                                     3.43
Package, The (1989)                                3.43
Nick of Time (1995)                                3.41
Blind Fury (1989)                                  3.41
Breakdown (1997)                                   3.36
Blown Away (1994)                                  3.29


## Вывод

Сформировал Гибридную рекомендательную систему на основании массива `ml-latest`:

Сделал следующую работу:
+ Загрузил данные о фильмах, рейтингах и тегах, и подготовил данные для обучения модели.
+ Обучил две модели:
  + `SVD` с `RMSE` `0.868`
  + `SVDpp` с `RMSE` `0.858`
+ На основании модели `SVD` имеющей более высокий `RMSE` спрогнозировал оценку фильма для тех которые пользователь еще не смотрел.
+ Для поиска похожих фильмов на основании жанров, подготовил данные для поиска ближайших соседей, жанры преобразуются в `TF-DF`, и модель `NearestNeighbors`.
+ Создал функцию для гибридной рекомендательной системы:
  + Функция комбинирует методы коллаборативной фильтрации `SVD` и контентной фильтрации `NearestNeighbors` для генерации рекомендаций для пользователя.
  + Функция использует последний просмотренный фильм
+ Вывел рекомендации для пользователя с `id` = `5`
